## Finding Patterns in Dissents (2007 onwards)

### Question
Are there common patterns in speeches and other features of the economy during dissents / for dissenters?

### Approach

1. Find dissenters from all FOMC meetings after 2007.
2. Find and download speeches made by these dissenters from the Fed website dating between 2 FOMC meetings before their first dissent and upto the FOMC meeting of their last dissent. These will be our considered 'documents'.
3. Run Topic modelling on these speeches together, using LDA (Latent Dirichlet Allocation).
4. Cluster the data based on topics, generate heat maps for topics frequently spoken about (unsupervised learning).
5. Repeat steps 3 and 4, but for speeches of dissenters who wanted tighter, easier and indeterminate action respectively.
5. Train LDA to predict governer action based on speeches made (supervised learning).

We will import the pandas libary to manipulate our datasets.

In [13]:
import pandas as pd

Let's import the dataset for effective Fed Funds Rate, and display the first 5 rows.'

In [109]:
fed_funds = pd.read_csv("datasets/FEDFUNDS.csv")
fed_funds.head()

,DATE,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.06
3,1954-10-01,0.85
4,1954-11-01,0.83


In [110]:
core_pce = pd.read_csv("datasets/PCEmonthly.csv")
core_pce.columns = list(core_pce.columns[0:1]) + ["EFFECTIVE_FFR"]
core_pce.head()

,DATE,EFFECTIVE_FFR
0,1959-02-01,1.71403
1,1959-03-01,1.81818
2,1959-04-01,-0.13066
3,1959-05-01,1.09027
4,1959-06-01,0.64711


In [38]:
fed_funds_range = pd.read_csv("datasets/FEDFUNDSrange.csv")
fed_funds_range.columns = list(fed_funds_range.columns[0:1]) + ["LOWERBOUND", "UPPERBOUND"]
fed_funds_range.head()

,DATE,LOWERBOUND,UPPERBOUND
0,2008-12-16,0.0,0.25
1,2008-12-17,0.0,0.25
2,2008-12-18,0.0,0.25
3,2008-12-19,0.0,0.25
4,2008-12-20,0.0,0.25


Importing all FOMC dissent data, sourced from publication ["Making Sense of Dissents: A History of FOMC Dissents." Federal Reserve Bank of St. Louis Review, Third Quarter 2014 Vol. 96, No. 3 by authors Daniel L. Thornton and David C. Wheelock](https://www.stlouisfed.org/fomcspeak/history-fomc-dissents).

In [133]:
fomc_dissents = pd.read_csv("datasets/FOMC_Dissents_Data.csv")
fomc_dissents.columns = fomc_dissents.iloc[2]
fomc_dissents
fomc_dissents.drop([0,1,2, 799, 800, 801, 802, 803], axis=0, inplace=True)
fomc_dissents = fomc_dissents.loc[:, fomc_dissents.columns.notnull()]

# filling NaN values
for col in fomc_dissents.columns[9:13]:
    fomc_dissents[col].fillna(0, inplace=True)

fomc_dissents["Year"] = pd.to_numeric(fomc_dissents["Year"]) # converting datatype to numeric
fomc_dissents["FOMC Meeting"] = pd.to_datetime(fomc_dissents["FOMC Meeting"]) # converting datatype to datetime
fomc_dissents.head()

2,Year,FOMC Meeting,Chair,Dissent (Y or N),FOMC Votes,Votes for Action,Votes Against Action,Number Governors Dissenting,Number Presidents Dissenting,No. Governors for Tighter,No. Governors for Easier,No. Presidents for Tighter,No. Presidents for Easier,Dissenters Tighter,Dissenters Easier,Dissenters Other/Indeterminate
3,1936,2036-03-19,Eccles,N,9,9,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1936,2036-05-25,Eccles,N,9,9,0,0,0,0,0,0,0,NaN,NaN,NaN
5,1936,2036-11-20,Eccles,N,11,11,0,0,0,0,0,0,0,NaN,NaN,NaN
6,1937,2037-01-26,Eccles,N,11,11,0,0,0,0,0,0,0,NaN,NaN,NaN
7,1937,2037-03-15,Eccles,N,8,8,0,0,0,0,0,0,0,NaN,NaN,NaN


Filtering for FOMC meetings when dissents took place.

In [134]:
dissenters = fomc_dissents.loc[fomc_dissents["Dissent (Y or N)"] == "Y"]
dissenters.head()

2,Year,FOMC Meeting,Chair,Dissent (Y or N),FOMC Votes,Votes for Action,Votes Against Action,Number Governors Dissenting,Number Presidents Dissenting,No. Governors for Tighter,No. Governors for Easier,No. Presidents for Tighter,No. Presidents for Easier,Dissenters Tighter,Dissenters Easier,Dissenters Other/Indeterminate
18,1938,2038-12-30,Eccles,Y,11,8,3,3,0,0,0,0,0,NaN,NaN,"Eccles, Ransom, Draper"
19,1939,2039-03-07,Eccles,Y,11,8,3,3,0,0,0,0,0,NaN,NaN,"Eccles, Ransom, Draper"
20,1939,2039-03-20,Eccles,Y,11,8,3,3,0,0,0,0,0,NaN,NaN,"Eccles, Ransom, Draper"
21,1939,2039-04-19,Eccles,Y,10,9,1,1,0,0,0,0,0,NaN,NaN,Draper
22,1939,2039-06-21,Eccles,Y,11,10,1,1,0,0,0,0,0,NaN,NaN,Draper


Filtering for meetings only after year 2007.

In [135]:
dissenters_post_2007 = dissenters[dissenters["Year"] > 2007]
dissenters_post_2007

2,Year,FOMC Meeting,Chair,Dissent (Y or N),FOMC Votes,Votes for Action,Votes Against Action,Number Governors Dissenting,Number Presidents Dissenting,No. Governors for Tighter,No. Governors for Easier,No. Presidents for Tighter,No. Presidents for Easier,Dissenters Tighter,Dissenters Easier,Dissenters Other/Indeterminate
710,2008,2008-01-21,Bernanke,Y,9,8,1,0,1,0,0,1,0,Poole,NaN,NaN
711,2008,2008-01-30,Bernanke,Y,10,9,1,0,1,0,0,1,0,Fisher,NaN,NaN
712,2008,2008-03-18,Bernanke,Y,10,8,2,0,2,0,0,2,0,"Fisher, Plosser",NaN,NaN
713,2008,2008-04-30,Bernanke,Y,10,8,2,0,2,0,0,2,0,"Fisher, Plosser",NaN,NaN
714,2008,2008-06-25,Bernanke,Y,10,9,1,0,1,0,0,1,0,Fisher,NaN,NaN
715,2008,2008-08-05,Bernanke,Y,11,10,1,0,1,0,0,1,0,Fisher,NaN,NaN
720,2009,2009-01-28,Bernanke,Y,9,8,1,0,1,0,0,0,0,NaN,NaN,Lacker
728,2010,2010-01-27,Bernanke,Y,10,9,1,0,1,0,0,1,0,Hoenig,NaN,NaN
729,2010,2010-03-16,Bernanke,Y,10,9,1,0,1,0,0,1,0,Hoenig,NaN,NaN
730,2010,2010-04-28,Bernanke,Y,10,9,1,0,1,0,0,1,0,Hoenig,NaN,NaN


In [141]:
# dataframe of dissenters for tighter action
dissenters_tighter = dissenters_post_2007[dissenters_post_2007["Dissenters Tighter"].notnull()]

# finding names of dissenters for tighter action
dissenter_names_tighter = set()
for dissenters in dissenters_tighter["Dissenters Tighter"]:
    diss = dissenters.split(",")
    diss = [d.strip() for d in diss]
    dissenter_names_tighter = dissenter_names_tighter.union(set(diss))

print("Dissenters for tighter action were", dissenter_names_tighter)
dissenters_tighter

Dissenters for tighter action were {'Fisher', 'Lacker', 'George', 'Rosengren', 'Kocherlakota', 'Poole', 'Mester', 'Hoenig', 'Plosser'}


2,Year,FOMC Meeting,Chair,Dissent (Y or N),FOMC Votes,Votes for Action,Votes Against Action,Number Governors Dissenting,Number Presidents Dissenting,No. Governors for Tighter,No. Governors for Easier,No. Presidents for Tighter,No. Presidents for Easier,Dissenters Tighter,Dissenters Easier,Dissenters Other/Indeterminate
710,2008,2008-01-21,Bernanke,Y,9,8,1,0,1,0,0,1,0,Poole,NaN,NaN
711,2008,2008-01-30,Bernanke,Y,10,9,1,0,1,0,0,1,0,Fisher,NaN,NaN
712,2008,2008-03-18,Bernanke,Y,10,8,2,0,2,0,0,2,0,"Fisher, Plosser",NaN,NaN
713,2008,2008-04-30,Bernanke,Y,10,8,2,0,2,0,0,2,0,"Fisher, Plosser",NaN,NaN
714,2008,2008-06-25,Bernanke,Y,10,9,1,0,1,0,0,1,0,Fisher,NaN,NaN
715,2008,2008-08-05,Bernanke,Y,11,10,1,0,1,0,0,1,0,Fisher,NaN,NaN
728,2010,2010-01-27,Bernanke,Y,10,9,1,0,1,0,0,1,0,Hoenig,NaN,NaN
729,2010,2010-03-16,Bernanke,Y,10,9,1,0,1,0,0,1,0,Hoenig,NaN,NaN
730,2010,2010-04-28,Bernanke,Y,10,9,1,0,1,0,0,1,0,Hoenig,NaN,NaN
731,2010,2010-06-23,Bernanke,Y,10,9,1,0,1,0,0,1,0,Hoenig,NaN,NaN


In [142]:
# dataframe of dissenters for easier action
dissenters_easier = dissenters_post_2007[dissenters_post_2007["Dissenters Easier"].notnull()]

# finding names of dissenters for easier action
dissenter_names_easier = set()
for dissenters in dissenters_easier["Dissenters Easier"]:
    diss = dissenters.split(",")
    diss = [d.strip() for d in diss]
    dissenter_names_easier = dissenter_names_easier.union(set(diss))

print("Dissenters for easier action were", dissenter_names_easier)
dissenters_easier

Dissenters for easier action were {'Kashkari', 'Bullard', 'Kocherlakota', 'Evans', 'Rosengren'}


2,Year,FOMC Meeting,Chair,Dissent (Y or N),FOMC Votes,Votes for Action,Votes Against Action,Number Governors Dissenting,Number Presidents Dissenting,No. Governors for Tighter,No. Governors for Easier,No. Presidents for Tighter,No. Presidents for Easier,Dissenters Tighter,Dissenters Easier,Dissenters Other/Indeterminate
742,2011,2011-11-02,Bernanke,Y,10,9,1,0,1,0,0,0,1,NaN,Evans,NaN
743,2011,2011-12-13,Bernanke,Y,10,9,1,0,1,0,0,0,1,NaN,Evans,NaN
755,2013,2013-06-19,Bernanke,Y,12,10,2,0,2,0,0,1,1,George,Bullard,NaN
759,2013,2013-12-18,Bernanke,Y,10,9,1,0,1,0,0,0,1,NaN,Rosengren,NaN
766,2014,2014-10-29,Yellen,Y,10,9,1,0,1,0,0,0,1,NaN,Kocherlakota,NaN
767,2014,2014-12-17,Yellen,Y,10,7,3,0,3,0,0,0,1,NaN,Kocherlakota,"Fisher, Plosser"
785,2017,2017-03-15,Yellen,Y,10,9,1,0,1,0,0,0,1,NaN,Kashkari,NaN
787,2017,2017-06-14,Yellen,Y,9,8,1,0,1,0,0,0,1,NaN,Kashkari,NaN
791,2017,2017-12-13,Yellen,Y,9,7,2,0,2,0,0,0,2,NaN,"Evans, Kashkari",NaN


In [143]:
# dataframe of dissenters for indeterminate action
dissenters_other = dissenters_post_2007[dissenters_post_2007["Dissenters Other/Indeterminate"].notnull()]

# finding names of dissenters for indeterminate action
dissenter_names_other = set()
for dissenters in dissenters_other["Dissenters Other/Indeterminate"]:
    diss = dissenters.split(",")
    diss = [d.strip() for d in diss]
    dissenter_names_other = dissenter_names_other.union(set(diss))

print("Dissenters for indetermnate action were", dissenter_names_other)
dissenters_other

Dissenters for indetermnate action were {'Fisher', 'Lacker', 'Kocherlakota', 'Plosser'}


2,Year,FOMC Meeting,Chair,Dissent (Y or N),FOMC Votes,Votes for Action,Votes Against Action,Number Governors Dissenting,Number Presidents Dissenting,No. Governors for Tighter,No. Governors for Easier,No. Presidents for Tighter,No. Presidents for Easier,Dissenters Tighter,Dissenters Easier,Dissenters Other/Indeterminate
720,2009,2009-01-28,Bernanke,Y,9,8,1,0,1,0,0,0,0,NaN,NaN,Lacker
740,2011,2011-08-09,Bernanke,Y,10,7,3,0,3,0,0,1,0,Kocherlakota,NaN,"Fisher, Plosser"
744,2012,2012-01-25,Bernanke,Y,10,9,1,0,1,0,0,0,0,NaN,NaN,Lacker
745,2012,2012-03-13,Bernanke,Y,10,9,1,0,1,0,0,0,0,NaN,NaN,Lacker
746,2012,2012-04-25,Bernanke,Y,10,9,1,0,1,0,0,0,0,NaN,NaN,Lacker
748,2012,2012-08-01,Bernanke,Y,12,11,1,0,1,0,0,0,0,NaN,NaN,Lacker
761,2014,2014-03-19,Yellen,Y,9,8,1,0,1,0,0,0,0,NaN,NaN,Kocherlakota
764,2014,2014-07-30,Yellen,Y,10,9,1,0,1,0,0,0,0,NaN,NaN,Plosser
765,2014,2014-09-17,Yellen,Y,10,8,2,0,2,0,0,0,0,NaN,NaN,"Fisher, Plosser"
767,2014,2014-12-17,Yellen,Y,10,7,3,0,3,0,0,0,1,NaN,Kocherlakota,"Fisher, Plosser"


In [147]:
print(dissenter_names_other, dissenter_names_tighter, dissenter_names_easier)

{'Fisher', 'Lacker', 'Kocherlakota', 'Plosser'} {'Fisher', 'Lacker', 'George', 'Rosengren', 'Kocherlakota', 'Poole', 'Mester', 'Hoenig', 'Plosser'} {'Kashkari', 'Bullard', 'Kocherlakota', 'Evans', 'Rosengren'}


In [11]:
import os

PATH_TO_SPEECHES = "./datasets/speeches/"
speeches = os.listdir(PATH_TO_SPEECHES)
speeches.sort()
speeches

['2008-01-08_Plosser_The_Economic_Outlook',
 '2008-02-06_Plosser_The_Economic_Outlook',
 '2008-03-03_Plosser_The_Benefits_of_Systemic_Monetary_Policy',
 '2008-03-28_Plosser_Foundations_for_Sound_Central_Banking',
 '2008-04-16_Plosser_Education_and_Economic_Prosperity',
 '2008-04-18_Plosser_Monetary_Policy_and_Financial_Stability',
 '2010-06-03_Hoenig_The_High_Cost_of_Exceptionally_Low_Rates',
 '2010-08-13_Hoenig_Hard_Choices',
 '2010-10-10_Hoenig_Not_Over_till_Over',
 '2010-10-12_Hoenig_Federal_Reserve_Mandate',
 '2010-11-05_Hoenig_Reforming_US_Housing_Finance',
 '2011-08-30_Kocherlakota_Communication_Credibility_Implementation',
 '2011-10-17_Evans_Fed_Dual_Mandate',
 '2011-12-05_Evans_Risk_Management',
 '2012-01-13_Lacker_Economic_Outlook',
 '2012-05-02_Lacker_Economic_Outlook',
 '2012-05-07_Lacker_Technology_Unemployment_Workforce_Development',
 '2012-09-18_Lacker_Maximum_Employment_Monetary_Policy',
 '2012-10-12_Lacker_Challenges_to_Economic_Growth',
 '2012-10-15_Lacker_Economic_Out

Now, importing the text from all the speeches we've gathered and creating a dataframe of the speeches.

In [38]:
documents = pd.DataFrame(columns=["Date", "Author",  "Document"])

for i, speech_filename in enumerate(speeches):
    speech_details = speech_filename.split("_")
    date = speech_details[0]
    author = speech_details[1]
    with open(PATH_TO_SPEECHES + speech_filename) as f:
        documents.loc[i] = [date, author, f.read()]

documents["Date"] = pd.to_datetime(documents["Date"])
documents

,Date,Author,Document
0,2008-01-08,Plosser,Introduction\n\nGood morning and thank you for...
1,2008-02-06,Plosser,Introduction\n\nIt is indeed a pleasure to be ...
2,2008-03-03,Plosser,Introduction\n\nGood morning. I am delighted t...
3,2008-03-28,Plosser,Introduction\n\nLet me thank the Global Interd...
4,2008-04-16,Plosser,Introduction\n\nGood afternoon. It is a pleas...
5,2008-04-18,Plosser,A couple of days ago I thought my role at this...
6,2010-06-03,Hoenig,1 I appreciate the opportunity to join you her...
7,2010-08-13,Hoenig,These are trying times for the U.S. economy. T...
8,2010-10-10,Hoenig,Introduction I was honored to be asked to spea...
9,2010-10-12,Hoenig,"Introduction and Framework Thank you, and it i..."


Dropping fischer's speeches post 2015 since he didn't dissent near those dates.

In [39]:
documents.drop(list(range(44, 50)), axis=0, inplace=True)
documents

,Date,Author,Document
0,2008-01-08,Plosser,Introduction\n\nGood morning and thank you for...
1,2008-02-06,Plosser,Introduction\n\nIt is indeed a pleasure to be ...
2,2008-03-03,Plosser,Introduction\n\nGood morning. I am delighted t...
3,2008-03-28,Plosser,Introduction\n\nLet me thank the Global Interd...
4,2008-04-16,Plosser,Introduction\n\nGood afternoon. It is a pleas...
5,2008-04-18,Plosser,A couple of days ago I thought my role at this...
6,2010-06-03,Hoenig,1 I appreciate the opportunity to join you her...
7,2010-08-13,Hoenig,These are trying times for the U.S. economy. T...
8,2010-10-10,Hoenig,Introduction I was honored to be asked to spea...
9,2010-10-12,Hoenig,"Introduction and Framework Thank you, and it i..."


### Preprocessing the data for LDA:

We will follow a tutorial for LDA located here: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

We will perform the following steps:

 - Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
 - Words that have fewer than 3 characters are removed.
 - All stopwords are removed.
 - Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
 - Words are stemmed — words are reduced to their root form.

In [45]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Pranav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [48]:
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [55]:
preprocess(documents["Document"][0]) # an example

['introduct',
 'good',
 'morn',
 'thank',
 'invit',
 'help',
 'kick',
 'year',
 'time',
 'year',
 'reflect',
 'prognost',
 'come',
 'economi',
 'import',
 'use',
 'exercis',
 'challeng',
 'task',
 'hand',
 'look',
 'help',
 'understand',
 'happen',
 'hand',
 'tell',
 'happen',
 'economist',
 'adept',
 'offer',
 'multipl',
 'explan',
 'economi',
 'behav',
 'hindsight',
 'term',
 'fact',
 'fuzzier',
 'come',
 'draw',
 'lesson',
 'futur',
 'clariti',
 'materi',
 'year',
 'later',
 'differ',
 'interpret',
 'offer',
 'midst',
 'immedi',
 'aftermath',
 'particular',
 'econom',
 'episod',
 'forecast',
 'futur',
 'cours',
 'difficult',
 'healthi',
 'dose',
 'humil',
 'appropri',
 'nonetheless',
 'forecast',
 'necessari',
 'task',
 'busi',
 'economi',
 'market',
 'product',
 'like',
 'evolv',
 'come',
 'year',
 'critic',
 'action',
 'today',
 'true',
 'policymak',
 'argu',
 'short',
 'morn',
 'want',
 'talk',
 'state',
 'economi',
 'share',
 'view',
 'econom',
 'outlook',
 'comment',
 'uncertai

In [53]:
processed_docs = documents["Document"].map(preprocess)
documents["Processed"] = processed_docs
documents.head()

,Date,Author,Document,Processed
0,2008-01-08,Plosser,Introduction\n\nGood morning and thank you for...,"[introduct, good, morn, thank, invit, help, ki..."
1,2008-02-06,Plosser,Introduction\n\nIt is indeed a pleasure to be ...,"[introduct, pleasur, birmingham, hometown, opp..."
2,2008-03-03,Plosser,Introduction\n\nGood morning. I am delighted t...,"[introduct, good, morn, delight, today, help, ..."
3,2008-03-28,Plosser,Introduction\n\nLet me thank the Global Interd...,"[introduct, thank, global, interdepend, center..."
4,2008-04-16,Plosser,Introduction\n\nGood afternoon. It is a pleas...,"[introduct, good, afternoon, pleasur, montgome..."
